# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [7]:
import pandas as pd
import numpy as np
from scipy import stats

In [8]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [70]:
# number of callbacks for black-sounding names
cb_b=sum(data[data.race=='b'].call)

# number of callbacks for white-sounding names
cb_w=sum(data[data.race=='w'].call)

#total resumes for bsn
tr_b=len(data[data.race=='b'])

#total resumes for wsn
tr_w=len(data[data.race=='w'])

#Proportion Analysis
b_sample=(cb_b/tr_b)
w_sample=(cb_w/tr_w)

#Pooled Proportion
pprop=((cb_b+cb_w)/(tr_b+tr_w))

print("Proportion of black sounding names= ",b_sample, "Proportion of white sounding names= ",w_sample)
print('Pooled proportion =', pprop)

Proportion of black sounding names=  0.06447638603696099 Proportion of white sounding names=  0.09650924024640657
Pooled proportion = 0.08049281314168377


In [66]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

In [12]:
#Check MD file

In [13]:
w = data[data.race=='w']
b = data[data.race=='b']

In [14]:
#Question3

In [76]:
#Calculate Margin of Error using Critical Value*Standard Deviation

sd=np.sqrt((b_sample*(1-b_sample)/tr_b)+(w_sample*(1-w_sample)/tr_w))

#At the 95% confidence interval, we know the z critical value = 1.96
z=1.96

me=z*sd

print("margin of error is = ", me)

#Calculating the CI
dif_prop=w_sample-b_sample

#CI = dif_prop+/- me
ci_lower=dif_prop-me
ci_upper=dif_prop+me

print("confidence interval is = ",(ci_lower, ci_upper))



margin of error is =  0.015255406349886438
confidence interval is =  (0.016777447859559147, 0.047288260559332024)


In [77]:
#Defining function for BS replicates

def bootstrap_replicate_1d(data, func):
    return func(np.random.choice(data, size=len(data)))

def draw_bs_reps(data, func, size=1):
    
    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)
    
        # Generate replicates
    for i in range(size):
            bs_replicates[i] = bootstrap_replicate_1d(data, func)
        
            return bs_replicates

In [78]:
#calculating mean difference
dif_mean= np.mean(w.call)-np.mean(b.call)

#calculating mean of data together
mean_count= np.mean(data.call)

#calculating shifted datasets
shifted_w= w.call - np.mean(w.call) + mean_count
shifted_b= b.call - np.mean(b.call) + mean_count

#generate separate bootstrap replicates
b_rep_w= draw_bs_reps(shifted_w, np.mean, 1000)
b_rep_b= draw_bs_reps(shifted_b, np.mean, 1000)

#generate bootstrap replicates
b_replicates = b_rep_w - b_rep_b

#calculate the p value
p = np.sum(b_replicates >= np.mean(w.call) - np.mean(b.call))/ len(b_replicates)
print('p-value =', p)

p-value = 0.0


C:\Users\Omar Baccus\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in greater_equal


In [79]:
#calculate confidence interval of bootstrap
conf_int=np.percentile(b_replicates, [2.5,97.5])

C:\Users\Omar Baccus\Anaconda3\lib\site-packages\numpy\lib\function_base.py:4291: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


In [80]:
#Calculating Margin of Error
b_replicates_me=stats.sem(b_replicates)
print("Margin of Error  =", b_replicates_me)

Margin of Error  = nan


In [81]:
#Grouping dataset by race and calculating mean
data.groupby('race', as_index=False).call.mean()

,race,call
0,b,0.064476
1,w,0.096509


In [82]:
mean_cb_w=0.096509
mean_cb_b=0.064476

In [83]:
#Grouping dataset by race and calculating variance
data.groupby('race', as_index=False).call.var()

,race,call
0,b,0.060344
1,w,0.087231


In [84]:
var_cb_w=0.087231
var_cb_b=0.060344

In [86]:
#Importing Module
from scipy.stats import ttest_ind, ttest_ind_from_stats
#Computing t-test of 2 sample means, with null hyp assuming they are equal
t1, p1 = ttest_ind_from_stats(mean_cb_w, np.sqrt(var_cb_w), 1000,
                              mean_cb_b, np.sqrt(var_cb_b), 1000,
                              equal_var=False)
print(t1, p1)

2.6368851703591814 0.008434100361904784


<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

In [ ]:
#Check MD